In [19]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [20]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [21]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column in sample.txt

In [22]:
from sentence_transformers import SentenceTransformer

In [23]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

C:\Users\SIYA RAM\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
vectors.shape

(8, 768)

In [25]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [26]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [27]:
index.add(vectors)

In [28]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000024E73032F70> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [29]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

#change into 2d vector

In [30]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

### Step 5: Search for similar vector in the FAISS index created 

In [32]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844839, 1.4039093]], dtype=float32)

In [33]:
I


array([[3, 2]], dtype=int64)

In [34]:
I.tolist()

[[3, 2]]

In [35]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [36]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [37]:
search_query

'I want to buy a polo t-shirt'